# EECE 5639 - Project 1

Matin Raayai Ardakani

Tony Smoragiewicz

Northeastern University

Fall 2021

In [6]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import tqdm
from scipy.ndimage import convolve, median_filter

### Dataset pre-processing

In [7]:
"""
Loads in images in a single folder and converts them to gray scale.
The images are returned as a python list
"""
def load_images_from_folder(folder):
    images = []
    for filename in tqdm.tqdm(sorted(os.listdir(folder))):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            gray = np.sum(img, 2) / 3
            images.append(gray)
    return images

In [8]:
office_dataset_path = "../Datasets/Office/"
red_chair_dataset_path = "../Datasets/RedChair"
office = np.array(load_images_from_folder(office_dataset_path))
red_chair = np.array(load_images_from_folder(red_chair_dataset_path))

print(f"Office dataset shape: {office.shape}")
print(f"Red chair dataset shape: {red_chair.shape}")

100%|██████████| 353/353 [00:00<00:00, 354.07it/s]


Office dataset shape: (1070, 240, 320)
Red chair dataset shape: (353, 240, 320)


### Filtering and Plotting Functions

In [ ]:
def convolve_temporal_filter(images, temp_filter):
    temp_filter = temp_filter.reshape(-1, 1, 1)
    return convolve(images.astype(np.float64), temp_filter)

def apply_median_filter(images, kernel):
    return median_filter(images, kernel)
    # # Temporal median filter
    # temp_median = np.zeros((idx-1, r, c))
    # for i in range(idx-5):
    #     median5 = [denoise[i,:,:], denoise[i+1,:,:], denoise[i+2,:,:], denoise[i+3,:,:], denoise[i+4,:,:]]
    #     median5 = np.array(median5)
    #     median5 = np.median(median5,axis=0)
    #     temp_median[i,:,:] = median5

def abs_and_thres(masks, min_thres=0, max_thres=500):
    masks = np.abs(masks)
    return np.select([masks > min_thres, masks < max_thres],
                 [np.ones_like(masks), np.zeros_like(masks)])

def convolve_spatial_filter(images, spatial_filter):
    spatial_filter = spatial_filter.reshape(1,
                                            spatial_filter.shape[0],
                                            spatial_filter.shape[1])
    return convolve(images, spatial_filter)

### Spatial and Temporal Filters

In [ ]:
naive_temp_filter = 0.5 * np.array([-1.,0.,1.])

frame = 215
frames = [images[frame-1,:,:],images[frame,:,:],images[frame+1,:,:]]
im_mask = np.tensordot(temporal_filter,frames,axes=(0,0))

plt.imshow(im_mask, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
# Spatial median filter
denoise = np.zeros((idx-1, r, c))
for i in range(idx-1):
    #denoise[i,:,:] = cv2.filter2D(src=images[i,:,:], ddepth=-1, kernel=gaussian)
    denoise[i,:,:] = cv2.medianBlur(images[i,:,:], 3)

In [ ]:
plt.subplot()
im = np.random.randint(idx)
plt.imshow(temp_median[im,:,:], cmap='gray', vmin=0, vmax=255)
print(im)

In [ ]:
# Temporal gradient
motion = np.zeros((idx-1, r, c))
for i in range(idx-1):
    temp_grad = denoise[i,:,:] - temp_median[i-1,:,:]
    #temp_grad = 0.5*(denoise[i+1,:,:] - denoise[i-1,:,:])
    motion[i,:,:] = np.abs(temp_grad)
    motion[i,:,:] = temp_grad**2/255
noise_line  = np.reshape(motion, ((idx-1)*r*c, 1) )

plt.subplot()
plt.hist(noise_line, bins=255)
plt.ylim([0,100000])
plt.xlim([1,255])

In [ ]:
# Apply a threshold
mask = np.zeros((idx-1, r, c))
threshold = 5
for i in range(idx-1):
    mask[i] = (motion[i,:,:] > threshold)*images[i,:,:]
mask_line  = np.reshape(mask, ((idx-1)*r*c, 1) )

plt.subplot()
plt.hist(mask_line, bins=255)
plt.ylim([0,100000])
plt.xlim([1,255])

In [ ]:
# Tough Images
plt.subplot()
im = 188
plt.imshow(motion[im,:,:], cmap='gray', vmin=0, vmax=255)
print(im)
print(np.amax(motion[im,:,:]))

In [ ]:
plt.subplot()
im = np.random.randint(idx)
im = 62
plt.imshow(mask[im,:,:], cmap='gray', vmin=0, vmax=255)
print(im)
print(np.amax(mask[im,:,:]))

In [ ]:
im = np.random.randint(idx)
im = 62
print(im)
plt.subplot()
plt.imshow(images[im,:,:], cmap='gray', vmin=0, vmax=255)
plt.imshow(mask[im,:,:], cmap='jet', alpha=0.5, vmin=0, vmax=255)